# Notebook for reporting the results 

In [2]:
import json
from Utils import evaluate_scenes,generate_summary_single,calculate_metrics,evaluate_f1_summaries
import pandas as pd 
import h5py
import numpy as np
import pickle
from sklearn.metrics import jaccard_score
import math

pd.set_option("display.precision", 3)
import os 


In [3]:
file = h5py.File('Data/original/googlenet_summe.h5')
gt_dicts ={}
gt_dicts['summe'] = {key: file[key]['change_points'][...] for key in list(file.keys())}
file = h5py.File('Data/original/googlenet_tvsum.h5')
gt_dicts['tvsum'] = {key: file[key]['change_points'][...] for key in list(file.keys())}
autoshot_annotations = pickle.load(open('gt_scenes_dict_baseline_v2.pickle','rb'))

In [4]:
def eval_json(json_file,gt_dict):
    json_dict = json.load(open(json_file,'r'))
    scores_dict = {}
    score_avg = []
    count_missing = 0
    for key in list(gt_dict.keys()):
      try:
        _,_,score,_ = evaluate_scenes(np.array(gt_dict[key]),np.array(json_dict[key]))
      except KeyError:
         count_missing+=1 
         print(f'This Key: {key} is missing')
         scores_dict[key] = -9999
         continue
      scores_dict[key] = score
      score_avg.append(score)
    scores_dict['AverageF1'] = np.mean(score_avg,where = score_avg!=-9999).item()
    return scores_dict

KTS algorithm

In [69]:
datasets = ['tvsum','summe']

features = ['densenet','vit','resnet','googlenet','fisher']
c_param = [7.5,5,2.5,1,0.8]
for dataset in datasets:
    for feat in features:
        json_file_name = f'Statistics/{dataset}/Results_{dataset}_{feat}.json'
        feature_dict = {}
        for c in c_param:
            json_shot_boundares = f'Results/kts/{dataset}/Shot_Boundaries_{dataset}_{feat}_{c}.json'
            if not os.path.exists(json_shot_boundares):
                print(f'Missing results {dataset} {feat} and {c}: Skip for now')
                
                score_dict = {}
                continue

            score_dict =  eval_json(json_shot_boundares,gt_dicts[dataset])
            feature_dict[c] = score_dict
        with open(json_file_name,'w') as file:
            json.dump(feature_dict,file,indent = 2)


In [70]:
result_data_frame = {}
dataset = 'tvsum'
for c in c_param:
    feat_dict ={}
    for feat in features:
        results = json.load(open(f'Statistics/{dataset}/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]['AverageF1']
    result_data_frame[c] = feat_dict 




In [71]:
results_dict = pd.DataFrame.from_dict(result_data_frame,orient = 'columns')

In [72]:
results_dict

,7.5,5.0,2.5,1.0,0.8
densenet,0.651,0.633,0.498,0.256,0.223
vit,0.605,0.635,0.539,0.318,0.289
resnet,0.657,0.624,0.506,0.276,0.240
googlenet,0.497,0.611,0.556,0.325,0.283
fisher,0.010,0.010,0.012,0.064,0.064


In [11]:
result_data_frame = {}
dataset = 'summe'
for c in c_param:
    feat_dict ={}
    for feat in features:
        results = json.load(open(f'Statistics/{dataset}/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]['AverageF1']
    result_data_frame[c] = feat_dict 

In [12]:
results_dict = pd.DataFrame.from_dict(result_data_frame,orient = 'columns')
results_dict

,7.5,5.0,2.5,1.0,0.8
densenet,0.188,0.211,0.229,0.167,0.154
vit,0.133,0.178,0.249,0.196,0.177
resnet,0.163,0.209,0.217,0.175,0.160
googlenet,0.132,0.152,0.186,0.181,0.166
fisher,0.018,0.003,0.018,0.058,0.058


Assign max val to each column, alongside value 


In [13]:
empty_frame = pd.DataFrame(columns = ["densenet",'vit','resnet','googlenet','c'],index=[f"video_{i}" for i in range(1,51)])

In [14]:
features =  ['densenet','vit','resnet','googlenet','fisher']
c_param = [7.5,5,2.5,1,0.8]
dataframe_list = []
dataset = 'tvsum'
for c in c_param:
    for feat in features:
        results = json.load(open(f'Statistics/{dataset}/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]
    c_2_5_data = pd.DataFrame.from_dict(feat_dict,orient = 'columns')
    c_2_5_data['highest_value']= c_2_5_data.apply(lambda x: max(x), axis=1)
    c_2_5_data['feature'] = c_2_5_data.apply(lambda x: c_2_5_data.columns[np.argmax(x)], axis = 1)
    c_2_5_data['reg_strength'] = c
    c_2_5_data.drop(columns=features,inplace=True)
    c_2_5_data.drop(['AverageF1'],axis=0,inplace=True)
    dataframe_list.append(c_2_5_data)



In [15]:
final_frame = pd.concat(dataframe_list)

In [16]:
empty_frame = pd.DataFrame(columns = ["highest_value","feature",'reg_strength'],index=[f"video_{i}" for i in range(1,51)])


In [17]:
for index in list(empty_frame.index):
    print(final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values)
    for values in final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values:
        empty_frame.loc[index] = values
    

[[0.588235294117647 'vit' 5.0]]
[[0.8363636363636364 'googlenet' 7.5]]
[[0.5930232558139535 'densenet' 5.0]]
[[0.8200000000000001 'resnet' 7.5]]
[[0.9302325581395349 'resnet' 5.0]]
[[0.5066666666666667 'densenet' 2.5]]
[[0.8771929824561403 'vit' 7.5]]
[[0.4583333333333333 'densenet' 5.0]]
[[0.6666666666666667 'densenet' 7.5]]
[[0.9454545454545454 'densenet' 7.5]
 [0.9454545454545454 'densenet' 5.0]]
[[0.6122448979591837 'googlenet' 5.0]]
[[0.5503355704697986 'densenet' 7.5]]
[[0.8837209302325583 'vit' 5.0]]
[[0.8695652173913043 'densenet' 7.5]]
[[0.5818181818181818 'densenet' 2.5]]
[[0.9007633587786259 'googlenet' 2.5]]
[[0.904109589041096 'resnet' 5.0]]
[[0.6545454545454547 'resnet' 5.0]]
[[0.5538461538461538 'resnet' 7.5]]
[[0.8500000000000001 'densenet' 7.5]]
[[0.787878787878788 'resnet' 7.5]]
[[0.8732394366197184 'resnet' 7.5]]
[[0.85 'densenet' 7.5]]
[[0.9 'googlenet' 5.0]]
[[0.9047619047619047 'vit' 7.5]]
[[0.7555555555555555 'googlenet' 5.0]]
[[0.9178082191780821 'densenet' 7.5]

In [18]:
empty_frame['highest_value'].mean()

0.7189176317016979

In [19]:
empty_frame[['feature','reg_strength']].value_counts()

feature    reg_strength
resnet     7.5             9
densenet   5.0             8
           7.5             8
googlenet  5.0             7
vit        5.0             4
           7.5             4
resnet     5.0             3
densenet   2.5             2
googlenet  7.5             2
           2.5             1
resnet     2.5             1
vit        2.5             1
Name: count, dtype: int64

In [27]:
features =  ['densenet','vit','resnet','googlenet','fisher']
c_param = [7.5,5,2.5,1,0.8]
dataframe_list = []
dataset = 'summe'
for c in c_param:
    for feat in features:
        results = json.load(open(f'Statistics/{dataset}/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]
    c_2_5_data = pd.DataFrame.from_dict(feat_dict,orient = 'columns')
    c_2_5_data['highest_value']= c_2_5_data.apply(lambda x: max(x), axis=1)
    c_2_5_data['feature'] = c_2_5_data.apply(lambda x: c_2_5_data.columns[np.argmax(x)], axis = 1)
    c_2_5_data['reg_strength'] = c
    c_2_5_data.drop(columns=features,inplace=True)
    c_2_5_data.drop(['AverageF1'],axis=0,inplace=True)
    dataframe_list.append(c_2_5_data)


In [28]:
final_frame = pd.concat(dataframe_list)

In [29]:
empty_frame = pd.DataFrame(columns = ["highest_value","feature",'reg_strength'],index=[f"video_{i}" for i in range(1,26)])
for index in list(empty_frame.index):
    for values in final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values:
        empty_frame.loc[index] = values


In [30]:
empty_frame[['feature','reg_strength']].value_counts()

feature    reg_strength
densenet   2.5             4
vit        2.5             4
densenet   7.5             3
resnet     2.5             3
googlenet  1.0             2
resnet     5.0             2
densenet   0.8             1
           5.0             1
googlenet  0.8             1
           2.5             1
resnet     1.0             1
vit        0.8             1
           1.0             1
Name: count, dtype: int64

In [31]:
empty_frame['highest_value'].mean()

0.3529050076376246

Creation of Summaries using shot boundaries produced by previous steps

In [40]:
datasets = ['tvsum','summe']

features = ['densenet','vit','resnet','googlenet']
c_param = [7.5,5,2.5,1,0.8]
for dataset in datasets:
    data_file = h5py.File(f'Data/original/googlenet_{dataset}.h5')
    for feat in features:
        json_file_name = f'Final_Summaries/Summaries_{dataset}_{feat}.json'
        summary_dict = {}
        for c in c_param:
            sum_c_dict = {}
            if not os.path.exists(f'Results/kts/{dataset}/Shot_Boundaries_{dataset}_{feat}_{c}.json'):
                print(f'Missing results {dataset} {feat} and {c}: Skip for now')
                continue
            json_shot_boundares = json.load(open(f'Results/kts/{dataset}/Shot_Boundaries_{dataset}_{feat}_{c}.json','r'))
            for key in list(json_shot_boundares.keys()):
                gtscore = data_file[key]['gtscore'][...]
                n_frame = data_file[key]['n_frames'][...]
                picks = data_file[key]['picks'][...]
                sum_c_dict[key] = generate_summary_single(json_shot_boundares[key],gtscore,n_frames = n_frame,positions=picks).tolist()
            summary_dict[c] = sum_c_dict
        with open(json_file_name,'w') as file:
            json.dump(summary_dict,file,indent = 2)
            
            

c:\VidsumMethods\Utils\summary_utils.py:28: RuntimeWarning: Mean of empty slice.
  shot_imp_scores.append((frame_scores[shot[0]:shot[1] + 1].mean()).item())
c:\Users\P70088666\Video-Learning\video-learning\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Analyse predictions 

In [34]:
all_c = {}
dataset = 'summe'
features = ['densenet','vit','resnet','googlenet']

c_param = [7.5,5,2.5,1,0.8]
for c in c_param:
    for i in range(len(features)-1):
        main_summaries = json.load(open(f'Final_Summaries/Summaries_{dataset}_{features[i]}.json'))
        feat_dict = {}
        for j in range(i+1,len(features)):
            new_summaries = json.load(open(f'Final_Summaries/Summaries_{dataset}_{features[j]}.json'))
            per_feat = {}

            for key in new_summaries['5'].keys():
                f1_score = jaccard_score(np.array(main_summaries[str(c)][key]),np.array(new_summaries[str(c)][key]))
                per_feat[key] = f1_score.item()
            feat_dict[f'{features[i]}_{features[j]}'] = per_feat
        all_c[c] = feat_dict


    

c:\Users\P70088666\Video-Learning\video-learning\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\P70088666\Video-Learning\video-learning\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\P70088666\Video-Learning\video-learning\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
import math
consistency_results = {}
for c in c_param:
    avg_score = []
    score_dict = {'mean':0,'var':0}
    for key in list(all_c[c].keys()):
        for video in list(all_c[c][key]):
            if math.isnan(all_c[c][key][video]):
                avg_score.append(0)
            else:
                avg_score.append(all_c[c][key][video])
    score_dict['mean'] = np.mean(avg_score).item()
    score_dict['var'] = np.var(avg_score).item()
    consistency_results[c] = score_dict 

In [54]:
consistency_results

{7.5: {'mean': 0.4304026374459701, 'var': 0.0978778296575062},
 5: {'mean': 0.45491277569516986, 'var': 0.07113286267962245},
 2.5: {'mean': 0.4660138199161519, 'var': 0.02981984796116382},
 1: {'mean': 0.5452901989900382, 'var': 0.023265368481604355},
 0.8: {'mean': 0.5726794863757704, 'var': 0.02001692791473999}}

In [ ]:
all_c = {}
dataset = 'tvsum'
c_param = [7.5,5,2.5,1,0.8]
for c in c_param:
    for i in range(len(features)-1):
        main_summaries = json.load(open(f'Final_Summaries/Summaries_{dataset}_{features[i]}.json'))
        feat_dict = {}
        for j in range(i+1,len(features)):
            new_summaries = json.load(open(f'Final_Summaries/Summaries_{dataset}_{features[j]}.json'))
            per_feat = {}

            for key in new_summaries['5'].keys():
                f1_score = jaccard_score(np.array(main_summaries[str(c)][key]),np.array(new_summaries[str(c)][key]))
                per_feat[key] = f1_score.item()
            feat_dict[f'{features[i]}_{features[j]}'] = per_feat
        all_c[c] = feat_dict

In [ ]:
consistency_results = {}
for c in c_param:
    avg_score = []
    score_dict = {'mean':0,'var':0}
    for key in list(all_c[c].keys()):
        for video in list(all_c[c][key]):
            if math.isnan(all_c[c][key][video]):
                avg_score.append(0)
            else:
                avg_score.append(all_c[c][key][video])
    score_dict['mean'] = np.mean(avg_score).item()
    score_dict['var'] = np.var(avg_score).item()
    consistency_results[c] = score_dict 

In [ ]:
consistency_results

## Autoshot


PELT

In [42]:
dataset = 'autoshot'

features = ['densenet','vit','resnet','googlenet']
c_param =[1,2,5,10,25,50]
for feat in features:
    json_file_name =f'Statistics/autoshot/pelt/Results_{dataset}_{feat}.json'
    feature_dict = {}
    for c in c_param:
        json_shot_boundares = f'Results/pelt/{dataset}/autoshot_shot_boundaries_{feat}_beta_{c}.json'
        print(f'Results/pelt/{dataset}/autoshot_shot_boundaries_{feat}_beta_{c}.json')
        if not os.path.exists(json_shot_boundares):
            print(f'Missing results {dataset} {feat} and {c}: Skip for now')
            score_dict = {}
            continue
        score_dict =  eval_json(json_shot_boundares,autoshot_annotations)
        feature_dict[c] = score_dict
    with open(json_file_name,'w') as file:
        json.dump(feature_dict,file,indent = 2)


Results/pelt/autoshot/autoshot_shot_boundaries_densenet_beta_1.json
This Key: 33338203782 is missing
This Key: 33803189290 is missing
This Key: 34996642719 is missing
This Key: 35087786411 is missing
This Key: 35219009690 is missing
This Key: 35460496768 is missing
This Key: 13335596295 is missing
This Key: 17331353453 is missing
This Key: 18592632588 is missing
This Key: 19079860649 is missing
This Key: 20915607770 is missing
This Key: 21267401290 is missing
This Key: 24881886015 is missing
This Key: 25545843427 is missing
This Key: 26922743731 is missing
This Key: 27107740997 is missing
This Key: 27109494185 is missing
This Key: 27438014338 is missing
This Key: 28251171415 is missing
This Key: 28541329142 is missing
This Key: 28582983201 is missing
This Key: 28855524125 is missing
This Key: 29015928124 is missing
This Key: 29021603968 is missing
This Key: 29047239060 is missing
This Key: 29513386332 is missing
This Key: 30325167905 is missing
This Key: 30382060590 is missing
This Key

In [51]:
#average results for each feature extractor separately
result_data_frame = {}
dataset = 'autoshot'
c_param = [1,2,5,10,25]
features = ['googlenet','resnet','densenet','vit']
for feat in features:

    feat_dict ={}
    for c in c_param:
        results = json.load(open(f'Statistics/autoshot/pelt/Results_{dataset}_{feat}.json','r'))
        feat_dict[c] = results[str(c)]['AverageF1']
    result_data_frame[feat] = feat_dict
results_dict = pd.DataFrame.from_dict(result_data_frame)

In [52]:
results_dict

,googlenet,resnet,densenet,vit
1,0.234,0.235,0.241,0.243
2,0.319,0.331,0.332,0.331
5,0.496,0.501,0.504,0.502
10,0.593,0.596,0.600,0.607
25,0.490,0.491,0.486,0.493


In [53]:
dataset = 'autoshot'
c_param = [1,2,5,10,25]
features = ['googlenet','resnet','densenet','vit']
dataframe_list = []

for c in c_param:
    feat_dict ={}
    for feat in features:
        results = json.load(open(f'Statistics/autoshot/pelt/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]
    c_2_5_data = pd.DataFrame.from_dict(feat_dict,orient = 'columns')
    c_2_5_data['highest_value']= c_2_5_data.apply(lambda x: max(x), axis=1)
    c_2_5_data['feature'] = c_2_5_data.apply(lambda x: c_2_5_data.columns[np.argmax(x)], axis = 1)
    c_2_5_data['reg_strength'] = c
    c_2_5_data = c_2_5_data[c_2_5_data['highest_value']!=-9999]
    c_2_5_data.drop(columns=features,inplace=True)

    c_2_5_data.drop(['AverageF1'],axis=0,inplace=True)
    dataframe_list.append(c_2_5_data)

In [54]:
final_frame = pd.concat(dataframe_list)

In [55]:
result_data_frame = {}
dataset = 'autoshot'
c_param = [1,2,5,10,25]
features = ['googlenet','resnet','densenet','vit']
for feat in features:

    feat_dict ={}
    for c in c_param:
        results = json.load(open(f'Statistics/autoshot/pelt/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]
        result_data_frame[c] = feat_dict 
results_dict = pd.DataFrame.from_dict(result_data_frame)

In [56]:
empty_frame = pd.DataFrame(columns = ["highest_value","feature",'reg_strength'],index=list(dataframe_list[0].index))
for index in list(empty_frame.index):
    print(final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values)
    print(final_frame.loc[index])
    max_values = final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values
    empty_frame.loc[index] = max_values[0]

[[0.9090909090909091 'googlenet' 10]]
             highest_value    feature  reg_strength
31602670982          0.154  googlenet             1
31602670982          0.286  googlenet             2
31602670982          0.588  googlenet             5
31602670982          0.909  googlenet            10
31602670982          0.649  googlenet            25
[[0.9090909090909091 'googlenet' 10]]
             highest_value    feature  reg_strength
38667643656          0.154  googlenet             1
38667643656          0.286  googlenet             2
38667643656          0.588  googlenet             5
38667643656          0.909  googlenet            10
38667643656          0.649  googlenet            25
[[0.9090909090909091 'googlenet' 5]]
             highest_value    feature  reg_strength
40355287299          0.286  googlenet             1
40355287299          0.500  googlenet             2
40355287299          0.909  googlenet             5
40355287299          0.750  googlenet            10
403

In [57]:
empty_frame['highest_value'].mean()

0.7772671812710692

KTS

In [73]:
dataset = 'autoshot'

features = ['densenet','vit','resnet','googlenet']
c_param = [7.5,5,2.5,1,0.8]
for feat in features:
    json_file_name = f'Statistics/autoshot/kts/Results_{dataset}_{feat}.json'
    feature_dict = {}
    for c in c_param:
        json_shot_boundares = f'Results/kts/{dataset}/Shot_Boundaries_{dataset}_{feat}_{c}.json'
        if not os.path.exists(json_shot_boundares):
            print(f'Missing results {dataset} {feat} and {c}: Skip for now')
            score_dict = {}
            continue
        score_dict =  eval_json(json_shot_boundares,autoshot_annotations)
        feature_dict[c] = score_dict
    with open(json_file_name,'w') as file:
        json.dump(feature_dict,file,indent = 2)

This Key: 33338203782 is missing
This Key: 33803189290 is missing
This Key: 34996642719 is missing
This Key: 35087786411 is missing
This Key: 35219009690 is missing
This Key: 35460496768 is missing
This Key: 13335596295 is missing
This Key: 17331353453 is missing
This Key: 18592632588 is missing
This Key: 19079860649 is missing
This Key: 20915607770 is missing
This Key: 21267401290 is missing
This Key: 24881886015 is missing
This Key: 25545843427 is missing
This Key: 26922743731 is missing
This Key: 27107740997 is missing
This Key: 27109494185 is missing
This Key: 27438014338 is missing
This Key: 28251171415 is missing
This Key: 28541329142 is missing
This Key: 28582983201 is missing
This Key: 28855524125 is missing
This Key: 29015928124 is missing
This Key: 29021603968 is missing
This Key: 29047239060 is missing
This Key: 29513386332 is missing
This Key: 30325167905 is missing
This Key: 30382060590 is missing
This Key: 30699682698 is missing
This Key: 32140458676 is missing
This Key: 

In [87]:
result_data_frame = {}
dataset = 'autoshot'
c_param = [7.5,5,2.5,1,0.8]
features = ['googlenet','resnet','densenet','vit']
for feat in features:

    feat_dict ={}
    for c in c_param:
        results = json.load(open(f'Statistics/autoshot/kts/Results_{dataset}_{feat}.json','r'))
        feat_dict[c] = results[str(c)]['AverageF1']
    result_data_frame[feat] = feat_dict 
results_dict = pd.DataFrame.from_dict(result_data_frame)

In [88]:
results_dict

,googlenet,resnet,densenet,vit
7.5,0.352,0.537,0.516,0.428
5.0,0.476,0.638,0.643,0.571
2.5,0.667,0.706,0.708,0.702
1.0,0.661,0.606,0.593,0.664
0.8,0.624,0.566,0.543,0.629


In [80]:
dataset = 'autoshot'
c_param = [7.5,5,2.5,1,0.8]
features = ['googlenet','resnet','densenet','vit']
dataframe_list = []

for c in c_param:
    feat_dict ={}
    for feat in features:
        results = json.load(open(f'Statistics/autoshot/kts/Results_{dataset}_{feat}.json','r')) 
        feat_dict[feat] = results[str(c)]
    c_2_5_data = pd.DataFrame.from_dict(feat_dict,orient = 'columns')
    c_2_5_data['highest_value']= c_2_5_data.apply(lambda x: max(x), axis=1)
    c_2_5_data['feature'] = c_2_5_data.apply(lambda x: c_2_5_data.columns[np.argmax(x)], axis = 1)
    c_2_5_data['reg_strength'] = c
    c_2_5_data = c_2_5_data[c_2_5_data['highest_value']!=-9999]
    c_2_5_data.drop(columns=features,inplace=True)

    c_2_5_data.drop(['AverageF1'],axis=0,inplace=True)
    dataframe_list.append(c_2_5_data)

In [81]:
final_frame = pd.concat(dataframe_list)

In [82]:
for feat in features:
        results = json.load(open(f'Statistics/autoshot/kts/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]
c_2_5_data = pd.DataFrame.from_dict(feat_dict,orient = 'columns')


In [83]:
empty_frame = pd.DataFrame(columns = ["highest_value","feature",'reg_strength'],index=list(dataframe_list[0].index))

In [84]:
for index in list(empty_frame.index):
    print(final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values)
    print(final_frame.loc[index])
    max_values = final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values
    empty_frame.loc[index] = max_values[0]

[[0.9600000000000001 'googlenet' 2.5]]
             highest_value    feature  reg_strength
31602670982          0.957     resnet           7.5
31602670982          0.957  googlenet           5.0
31602670982          0.960  googlenet           2.5
31602670982          0.923  googlenet           1.0
31602670982          0.889     resnet           0.8
[[1.0 'resnet' 7.5]
 [1.0 'resnet' 5.0]
 [1.0 'googlenet' 2.5]
 [1.0 'googlenet' 1.0]]
             highest_value    feature  reg_strength
38667643656           1.00     resnet           7.5
38667643656           1.00     resnet           5.0
38667643656           1.00  googlenet           2.5
38667643656           1.00  googlenet           1.0
38667643656           0.96        vit           0.8
[[0.8 'resnet' 2.5]]
             highest_value    feature  reg_strength
40355287299          0.286  googlenet           7.5
40355287299          0.500   densenet           5.0
40355287299          0.800     resnet           2.5
40355287299          

In [85]:
empty_frame['highest_value'].mean()

0.8150662415705904

In [ ]:
results_dict

In [ ]:
result_data_frame = {}
dataset = 'autoshot'
c_param = [1,2,5,10,25,50]
features = ['googlenet','resnet','densenet','vit']
for c in c_param:
    feat_dict ={}
    for feat in features:
        results = json.load(open(f'Statistics/autoshot/pelt/Results_{dataset}_{feat}.json','r'))
        feat_dict[feat] = results[str(c)]
    c_2_5_data = pd.DataFrame.from_dict(feat_dict,orient = 'columns')
    c_2_5_data['highest_value']= c_2_5_data.apply(lambda x: max(x), axis=1)
    c_2_5_data['feature'] = c_2_5_data.apply(lambda x: c_2_5_data.columns[np.argmax(x)], axis = 1)
    c_2_5_data['reg_strength'] = c
    c_2_5_data = c_2_5_data[c_2_5_data['highest_value']!=-9999]
    c_2_5_data.drop(columns=features,inplace=True)

    c_2_5_data.drop(['AverageF1'],axis=0,inplace=True)
    dataframe_list.append(c_2_5_data)

In [120]:
final_frame = pd.concat(dataframe_list)
empty_frame = pd.DataFrame(columns = ["highest_value","feature",'reg_strength'],index=list(dataframe_list[0].index))

In [121]:
for index in list(empty_frame.index):
    print(final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values)
    print(final_frame.loc[index])
    max_values = final_frame.loc[index][final_frame.loc[index].highest_value==final_frame.loc[index].highest_value.max()].values
    empty_frame.loc[index] = max_values[0]

[[0.9600000000000001 'googlenet' 2.5]]
             highest_value    feature  reg_strength
31602670982       0.285714     resnet          25.0
31602670982       0.857143   densenet          10.0
31602670982       0.956522  googlenet           5.0
31602670982       0.960000  googlenet           2.5
31602670982       0.923077  googlenet           1.0
31602670982       0.923077        vit           0.8
31602670982       0.888889     resnet           0.6
31602670982       0.153846  googlenet           1.0
31602670982       0.285714  googlenet           2.0
31602670982       0.588235  googlenet           5.0
31602670982       0.909091  googlenet          10.0
31602670982       0.648649  googlenet          25.0
31602670982       0.387097  googlenet          50.0
[[1.0 'resnet' 5.0]
 [1.0 'googlenet' 2.5]
 [1.0 'googlenet' 1.0]]
             highest_value    feature  reg_strength
38667643656       0.153846  googlenet          25.0
38667643656       0.909091     resnet          10.0
3866764365

In [122]:
empty_frame['highest_value'].mean()

np.float64(0.8277607918118278)